## Extras sobre funciones en Python



### Desempacar secuencias o diccionarios directamente como argumentos

Si ya tengo los parámetros que quiero pasar a una función, los "desempaco". 


In [43]:
def saludar(nombre, saludo="Hola", sufijo="¿qué tal?"):
    """Dado un nombre y, opcionalmente, un saludo y/o sufijo, devuelve 
    una cadena saludo + nombre + sufijo"""
    
    return "{} {}, {}".format(saludo, nombre, sufijo)  

print(saludar("Martín"))
print(saludar("Fernando", 'Ey'))
print(saludar("Lionel Messi", "Estimado", 'usted es un genio'))

Hola Martín, ¿qué tal?
Ey Fernando, ¿qué tal?
Estimado Lionel Messi, usted es un genio


In [44]:
otra_data = ('hola', '¿cómo va?')

saludo, sufijo = otra_data

saludar('José', saludo, sufijo)   # el primer elemento desempacado, va al primer arg posicional disponible... 


'hola José, ¿cómo va?'

In [45]:
saludar('José', *otra_data)

'hola José, ¿cómo va?'

In [54]:
mi_data = {'nombre': 'Cristian', 'saludo': 'Ey', 'sufijo': 'qué onda?', 'otra': 'color'}

saludar(**mi_data)     # Obviamente, si pasamos más parámetros de los esperados por la función (y no permite parámetros arbritrarios), dará un error

TypeError: saludar() got an unexpected keyword argument 'otra'

In [55]:
def saludar2(nombre, saludo="Hola", sufijo="¿qué tal?", **kwargs):
    """Dado un nombre y, opcionalmente, un saludo y/o sufijo, devuelve 
    una cadena saludo + nombre + sufijo"""
    
    return "{saludo} {nombre}, {sufijo}".format(**locals())  #la funcion locals() devuelve el diccionario de todos los objetos definidos en el espacio de nombres



In [56]:
saludar2(**mi_data)

'Ey Cristian, qué onda?'

### Generadores



Los generadores son similares a las funciones, pero permiten crear **una serie de resultados** para ser iterados (o sea, genera un iterador), devolviendo un valor por cada llamada. Ejemplos de funciones generadoras son `zip`, `enumerate`, `range` y `reversed`, que ya vimos. 

También mencionamos la versión por comprensión `(f(x) for x in iter)`. La forma funcional es casi igual a la las funciones comunes, pero en vez de `return` se utiliza `yield` que funciona como **una pausa** (devolviendo opcionalmente un valor) en la ejecución.


In [35]:
def generador_ejemplo():
    print('antes del primer yield')
    yield 1               # sale devolviendo. la proxima llamada comenzará en la siguiente linea
    print('antes del segundo')
    yield                 # como return, puede devolver None
    print('antes del último')
    yield 10
    print('final')

In [36]:
valores = generador_ejemplo()
valores

<generator object generador_ejemplo at 0x111dc3ad0>

Para pedirles los valores uno a uno a un iterador (un generador es siempre un iterador), podemos usar la función `next`

In [37]:
next(valores)

antes del primer yield


1

In [38]:
next(valores)

antes del segundo


In [39]:
next(valores)

antes del último


10

In [40]:
next(valores)

final


StopIteration: 

Que es básicamente lo que hace la sentencia `for`

In [41]:
for valor in generador_ejemplo():
    print('Valor: ', valor)

antes del primer yield
Valor:  1
antes del segundo
Valor:  None
antes del último
Valor:  10
final


La clave de un generador es que **no es necesario computar todos los valores posibles** de una serie, sino que los vamos creando uno a uno *bajo demanda*. Quizas antes de terminar la serie podemos dar por concluido el cómputo, y entonces habremos ahorrado tiempo de procesador y memoria. 

In [42]:
def fibonacci(n):
    """Generador de n primeros numeros de fibonacci"""
    i = 0
    a, b = 0, 1
    while i < n:
        i += 1
        yield a            # devolvemos un valor. En el proximo llamado retornará desde este punto, 
                           # con los valores de locals() tal como estaban antes de hacer el yield
        a, b = b, a + b

list(fibonacci(20))

[0,
 1,
 1,
 2,
 3,
 5,
 8,
 13,
 21,
 34,
 55,
 89,
 144,
 233,
 377,
 610,
 987,
 1597,
 2584,
 4181]

### Manejo de excepciones

Ya vimos que a veces suceden errores: por ejemplo, cuando apuntamos a un elemento mayor al tamaño de una secuencia, cuando pedimos el valor de una clave que no existe en un diccionario, cuando dividimos por cero, cuando intentamos un *casting de tipos* no válido, etc. 

No hay problema interactivamente, porque podemos corregir y reintentar (lo que es genial), pero muchas veces queremos o necesitamos "capturar" el potencial error o excepción, ya sea para subsanarlo de alguna manera, registrarlo o lanzar otro más específico en reemplazo, etc. 
   
La sintaxis es un poco parecida al `if / elif / else`

In [ ]:
int('diez')

In [ ]:
while True:
    try:
        x = int(input("Ingrese un número entero: "))
        print("qué lindo número el {}".format(x))
        break
    except ValueError:
        print("Eso no es un número válido.")

Una sintaxis más completa permite multiples bloques `except`, un mismo bloque except  un bloque `else` que se ejecuta cuando no se originó ninguna excepción y un bloque `finally` que se ejecuta siempre

In [ ]:

try:
    x = int(input("Ingrese el divisor: "))
    print(10/x)
except ZeroDivisionError:
    print("hubo un error de division por cero, obvio")
except ValueError:
    print("hubo un error de valor. Poné un numero! ")
else:
    print('todo salió bien. puedo hacer más operaciones')
finally:
    print('no sé qué pasó ni me interesa: yo me ejecuto igual')
    


¡Esto se usa mucho!  En la filosofía de Python, que espera comportamientos y no tipos,  es **mejor pedir perdón que pedir permiso**. Es decir, es preferible capturar potenciales errores de un intento de operación que verificar precondiciones. 
   

### Lectura y escritura de archivos

Siempre necesitamos leer y escribir archivos. Es la forma básica de interactuar con el resto del sistema, introducir y exportar datos para la "computación". Como en Python todo es un objeto, lo que tenemos es un "objeto manejador de archivos" . La forma más básica de obtener uno es con la función `open()` que se le dice la ruta al archivo y modo/s, que se especifican con 


       'r': lectura (default)
       'w': (sobre)escritura
       'a': agregar contenido al final 
       'x': para escribir, pero no sobreescribe si existe el path
       'b': modo binario
       't': modo texto (default)
       '+':	actualizar contenido
       



In [22]:
%%writefile archivo.txt
UN EJEMPLO DE TEXTO

CON MULTIPLES LINEAS

SI!

Overwriting archivo.txt


In [23]:
readme = open('archivo.txt')    # se usa modo default 'rt' (sólo lectura, formato texto)
print(readme)

<_io.TextIOWrapper name='archivo.txt' mode='r' encoding='UTF-8'>


Los objetos `file like` como `readme` tienen un método principal llamado `read` que lee `n` cantidad de caracteres (o bytes en modo binario) o todo el contenido del archivo si no se especifica

In [24]:
texto = readme.read()

texto

'UN EJEMPLO DE TEXTO\n\nCON MULTIPLES LINEAS\n\nSI!'

Atenti: el objeto manejador lleva internamente la **posición del cursor**. Por ejemplo, si invocan multiples veces el metodo `read()`, leerán porciones consecutivas del archivo. 

Métodos útiles: `read()`, `readlines()`, `write()`, `writelines()`

A veces simplemente queremos hacer algo "linea por linea". En vez de usar `readlines()` y cargar todo en memoria (que puede ser grande), podemos iterar directamente sobre el archivo, que nos devolverá una línea 

In [ ]:
readme = open('archivo.txt')
for linea in readme:
    print(linea[:-1].upper())

In [ ]:
readme = open('README.rst')
for i, linea in enumerate(readme):
    if i < 4:
        print(linea)
    else:
        break

In [ ]:
readme2 = open('archivo.txt', 'r')

print("".join(readme2.readlines()))


Hasta que no se invoca al método `close()` el archivo está manejado en memoria por Python (y puede causar conflictos si queremos abrir el archivo desde otro programa). Como los objetos `file` saben usar un bloque `with` (manejador de contexto), podemos usarlo para que se cierre automáticamente. 

In [ ]:
with open('README.rst', 'r') as readme:
    lineas = readme.readlines()[0:5]    #mete las líneas en una lista
print(''.join(lineas))


### Archivos binarios

La manipulación de archivos binarios es equivalente. 



![](../img/circle.bmp)



In [1]:
with open('../img/circle.bmp', 'rb') as img_file:
    img_data = img_file.read()
img_data

b'BM\x12\x14\x00\x00\x00\x00\x00\x00\x8a\x00\x00\x00|\x00\x00\x002\x00\x00\x002\x00\x00\x00\x01\x00\x10\x00\x03\x00\x00\x00\x88\x13\x00\x00\x13\x0b\x00\x00\x13\x0b\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xf8\x00\x00\xe0\x07\x00\x00\x1f\x00\x00\x00\x00\x00\x00\x00BGRs\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x02\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\x00\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xde\xff\xdb\xfeU\xfd\x8e\xfb\x08\xfa\xc3\xf8!\xf8!\xf8\xc3\xf8\x08\xfa\x8e\xfbU\xfd\xdb\xfe\xde\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\xff\

In [58]:
type(img_data)

bytes

Por ejemplo, podemos 

In [2]:
with open('../img/circle_edited.bmp', 'wb') as img_file: 
    img_file.write(img_data.replace(b'\xff', b'\x00'))

![](../img/circle_edited.bmp)

A veces, la información binaria es simplemente texto y sólo hace falta "decodificarlo" para que python lo convierta a un objeto unicode. Para eso se usa el método `decode()` al que se le pasa el tipo de "encoding" (latin-1, ascii, utf8, etc). 

In [40]:
img_data.decode('latin-1')[0:30]

'BM\x12\x14\x00\x00\x00\x00\x00\x00\x8a\x00\x00\x00|\x00\x00\x002\x00\x00\x002\x00\x00\x00\x01\x00\x10\x00'

esa representacion como texto de datos que no son texto no es muy util. Sin embargo a veces es util convertir a una representación hexadecimal

In [42]:
img_data.hex()[0:30]

'424d12140000000000008a0000007c'

Qué pasa si una estructura de datos binaria del archivo es compuesta? Por ejemplo, texto, números, etc.  Para eso existe el módulo `struct`

In [43]:
import struct 

values = (1, 'ab'.encode('utf-8'), 2.7)
s = struct.Struct('I 2s f')
packed_data = s.pack(*values)

print('Original values:', values)
print('Format string  :', s.format)
print('Uses           :', s.size, 'bytes')
print('Packed Value   :', packed_data.hex())

Original values: (1, b'ab', 2.7)
Format string  : b'I 2s f'
Uses           : 12 bytes
Packed Value   : 0100000061620000cdcc2c40


In [53]:
s.unpack(packed_data)

(1, b'ab', 2.700000047683716)